In [11]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_community.vectorstores import Chroma
from langchain.vectorstores.base import VectorStoreRetriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from typing import TypedDict, List
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import END, StateGraph
from transformers import pipeline

In [3]:
embedding = HuggingFaceEmbeddings()
vectorstore = Chroma(persist_directory='./database', embedding_function=embedding)

C:\Users\rebal\AppData\Local\Temp\ipykernel_30124\130061073.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory='./database', embedding_function=embedding)


In [15]:
retriever = vectorstore.as_retriever(search_kwargs={"k":5})

In [6]:
d = retriever.invoke('Anarchy')
d

[Document(metadata={'title': 'Bodmin'}, page_content='Rebellions'),
 Document(metadata={'title': 'Anarchism'}, page_content='References\n\nExplanatory notes\n\nCitations\n\nGeneral and cited sources\n\nPrimary sources\n\nSecondary sources\n\nTertiary sources\n\nFurther reading \n \n  \n  Criticism of philosophical anarchism.\n \n  A defence of philosophical anarchism, stating that "both kinds of \'anarchism\' [i.e. philosophical and political anarchism] are philosophical and political claims." (p.\xa0137)\n  Anarchistic popular fiction novel.\n \n \n \n  An argument for philosophical anarchism.\n\nExternal links \n\n \n Anarchy Archives – an online research center on the history and theory of anarchism.\n\n \nAnti-capitalism\nAnti-fascism\nEconomic ideologies\nFar-left politics\nLeft-wing politics\nLibertarian socialism\nLibertarianism\nPolitical culture\nPolitical ideologies\nPolitical movements\nSocial theories\nSocialism'),
 Document(metadata={'title': 'Albert Camus'}, page_content=

In [50]:
class GraphState(TypedDict):
    question: str
    documents: List[Document]
    answer: str

In [12]:
model = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [51]:
def retrieve(state):
        """
        Uses the query to match documents in the database that are similar to the question.
        """
        print('---RETRIEVE---')
        question = state['question']

        documents = retriever.invoke(question)
        return {'documents': documents, 'question': question}

def generate(state):
        """
        Generate the answer to the question.
        """
        print('---GENERATE---')
        question = state['question']
        documents = state['documents']
        context = '\n\n'.join([d.page_content for d in documents])

        answer = model(question = question, context = context)['answer']
        return {'documents': documents, 'question': question, 'answer': answer}

In [52]:
question = 'Who was Achilles'
documents = retriever.invoke(question)

In [53]:
# context = '\n\n'.join([d.page_content for d in documents])
# print(context)

In [54]:
# model(question = question, context = context)['answer']

In [55]:
workflow = StateGraph(GraphState)
workflow.add_node('retrieve', retrieve)
workflow.add_node('generate', generate)

workflow.set_entry_point('retrieve')
workflow.add_edge('retrieve', 'generate')
workflow.add_edge('generate', END)

graph = workflow.compile()

In [56]:
result = graph.invoke({"question": question})

---RETRIEVE---
---GENERATE---


In [58]:
result['answer']

'a hero of the Trojan War'